In [ ]:
from pathlib import Path
import os

import pandas as pd

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Physical model

For this example we propose a resistance/capacity approach.
 Based on electrical circuit analogy, each layer of the wall is modeled by two resistance and a capacity:

![image.png](Wall_model.png)

The following is a brief description of the thermal model, as it is not the scope of this document. See the <code>*mo</code> for full informations.

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $
    - $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $


- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.


- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $
    - With the surrounding considered to be at $ T_{ext} $


- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.


- $ T_{ext}$ and $T_{int} $ are boundary conditions


Initial conditions for the layers temperatures are taken from the measured data.
 It is assumed to be the mean temperature measured by the sensors on each face of a layer.
 In python and using modelica "object name", it can be written :

In [ ]:
init_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
}

We specify the simulation running options. As the initial condition, it is written
as a python dictionary.

In Modelica, <code>startTime</code> and <code>stopTime</code> correspond to the number
of seconds since the beginning of the year. The values can be found in the file created
earlier using <code>df_to_combitimetable</code>

<code>stepSize</code> is the simulation timestep size. In this case it's 5min or
300sec.

<code>tolerance</code> and <code>solver</code> are related to solver configuration
do not change if you don't need to.

In [ ]:
simulation_opt = {
        "startTime": 6912000,
        "stopTime": 7084500,
        "stepSize": 300,
        "tolerance": 1e-06,
        "solver": "dassl"
}

We can now define a modelitool <code>Simulator</code>. This object is designed
to handle modelica simulation and output post treatment. It will be used
in objects that automate simulation such as an <code>Identificator</code>
or a <code>SAnalysis</code> object.

In [ ]:
from modelitool.simulate import Simulator

In [ ]:
# Values in output list correspond to sensors name and value "T"
simu = Simulator(
    model_path=Path(TUTORIAL_DIR) / "etics_v0.mo",
    simulation_options=simulation_opt,
    init_parameters=init_dict,
    output_list=["T_coat_ins.T",
                 "T_ins_ins.T",
                 "Tw_out.T"],
)

From here, it is very simple to run a simulation using <code>simulate()</code>
method, and to get the results required in <code>output_list</code> using
<code>get_results()</code> method.

In [ ]:
simu.simulate()
initial_results = simu.get_results()

If we want to compare these results to the measures, we have to load the
boundary file.

*Note: since Modelica uses seconds, time axis in the rest of the document it will
 be in seconds. A conversion function might be integrated in next versions*

In [ ]:
#Pandas lines to load boundary file
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "boundary_temp.txt",
    skiprows=3,
    sep='\t',
    index_col=0,
    header=None,
)

reference_df.columns = ["T_ext", "Sol_rad", "T_Wall_Ins", "T_Ins_Ins",
                        "T_Ins_Coat", "T_int"]

temperature_columns = ["T_ext", "T_Wall_Ins", "T_Ins_Ins",
                        "T_Ins_Coat", "T_int"]

reference_df[temperature_columns] = reference_df[temperature_columns] + 273.15

In [ ]:
# Plotly lines
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=initial_results.index,
    y=initial_results["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x=reference_df.index,
    y=reference_df.T_Ins_Ins,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Model VS Reality : temperature between two layer of insulation',
    xaxis_title='Time [sec since 01/01]',
    yaxis_title='Temperature [K]')

fig.show()

Considering the above graphic, we could say that the model results are pretty bad.
- The difference between predicted and measured temperature reaches ~10K
- There seem to be a small "shift" between model and reality.
Reference temperature peaks are happening earlier than model peaks.

There can be a lot of causes to this discrepancy. From physical phenomenon approximation
to material physical properties values.

*A good Idea would be to plot the other outputs variables or to perform a heat
balance analysis. But remember that this is a tutorial to use modelitool :).*

*For now we will just go on, and perform a sensitivity analysis*

# Sensitivity analisys

This project aims at characterizing the insulation material thermal conductivity.
To do so, we want to find the value $ \lambda_{etics} $ that would minimise the discrepancy between
predicted temperature between 2 layers of insulation material and the sensors measures.

However, it is very important to know how $ \lambda_{etics} $ is important
to describe the error. Other assumptions, such as material thermal properties may
have a strong influence on the model prediction.

Therefore, we use the sensitivity analysis to "rank" the parameter by order of influence
on the error between measured temperature and model prediction.

The chosen error function is the Mean Square Error (RMSE):

$MSE = \frac{1}{1-N}\sum \limits_{i=1}^{N} (y_i - \hat y_i)^2$

The chosen parameters for the sensitivity analysis are listed below:
- Concrete thermal capacity <code>capa_concrete</code> with an uncertainty of 20%
- Concrete density <code>rho_concrete</code> with an uncertainty of 20%
- All layer thermal conductivity with an uncertainty af 20%
- The coating shortwave solar absorption coefficient <code>Alpha_clo</code>
with an uncertainty of 20%
- The inside and outside conductive/convective equivalent thermal resistance
with an uncertainty of 20%

In modelitool, these uncertainties must be described using a dictionary:

In [ ]:
modelitool_problem = {
    "capa_concrete": [1000-0.2*1000, 1000+0.2*1000],
    "rho_concrete": [875-0.2*875, 875+0.2*875],
    "lambda_concrete": [1-0.2*1, 1+0.2*1],
    "lambda_coating": [1-0.2*1, 1+0.2*1],
    "Lambda_ins.k": [0.04-0.2*0.04, 0.04+0.2*0.04],
    "Alpha_clo.k": [0.5-0.2*0.5, 0.5+0.2*0.5],
    "R_conv_ext.k": [0.04-0.2*0.04, 0.04+0.2*0.04],
    "R_conv_int.k": [0.13-0.2*0.13, 0.13+0.2*0.13],
}

We can now use a <code>SAnalysis</code> to set-up the study. We have to pass
the <code>Simulator</code> previously describe, along with the corresponding
 problem description. A Sensitivity Analysis is also required. In this case we choose Sobol
, as there is few uncertain parameter.

*Note: for now only <code>Sobol</code>, <code>Sobol</code>, <code>FAST</code>,
and <code>Morris</code> methods are implemented.*

In [ ]:
from modelitool.sensitivity import SAnalysis

sa_study = SAnalysis(
    simulator=simu,
    sensitivity_method="Sobol",
    parameters_config=modelitool_problem
)


We draw a sample of parameters to simulate. Each method has its sampling method.
Please see SALib documentation for further explanation (https://salib.readthedocs.io/en/latest/index.html)

In [ ]:
# Additional arguments can be passed. if arguments is not used it
# Uses default SALib configuration
sa_study.draw_sample(n=100, arguments={"calc_second_order": True})

The sample is available as a 2d array <code>sa_study.sample</code>. Lines are simulations
to run and columns are parameters values.

Let's run the simulations.

**CAREFUL depending on your computer, it can take a long time in my case 30'**

In [ ]:
sa_study.run_simulations()